## pack_padded_sequence


In [3]:
import torch
import torch.nn as nn
from torch.nn import utils as nn_utils

batch_size = 2
max_length = 3
hidden_size = 2
n_layers = 1


In [5]:
tensor_in = torch.FloatTensor([[1, 0, 0], [1, 2, 3]]).resize_(2, 3, 1)
seq_lengths = [1, 3]  # list of integers holding information about the batch size at each sequence step
print("seq_lengths:\n", seq_lengths)
print("tensor_in:\n", tensor_in)


seq_lengths:
 [1, 3]
tensor_in:
 tensor([[[1.],
         [0.],
         [0.]],

        [[1.],
         [2.],
         [3.]]])


In [ ]:
seq_lens = torch.IntTensor([1, 3])
_, idx_sort = torch.sort(seq_lens, dim=0, descending=True)
_, idx_unsort = torch.sort(idx_sort, dim=0)
# idx_sort
# idx_unsort

tensor([1, 0])

In [12]:
order_seq_lengths = torch.index_select(seq_lens, dim=0, index=idx_sort)
order_tensor_in = torch.index_select(tensor_in, dim=0, index=idx_sort)
print("order_seq_lengths:\n", order_seq_lengths)
print("order_tensor_in:\n", order_tensor_in)

order_seq_lengths:
 tensor([3, 1], dtype=torch.int32)
order_tensor_in:
 tensor([[[1.],
         [2.],
         [3.]],

        [[1.],
         [0.],
         [0.]]])


In [13]:
x_packed = nn_utils.rnn.pack_padded_sequence(order_tensor_in, order_seq_lengths, batch_first=True)
rnn = nn.RNN(1, hidden_size, n_layers, batch_first=True)
h0 = torch.randn(n_layers, batch_size, hidden_size)
y_packed, h_n = rnn(x_packed, h0)
print('x_packed:\n', x_packed)
print('y_packed:\n', y_packed)

x_packed:
 PackedSequence(data=tensor([[1.],
        [1.],
        [2.],
        [3.]]), batch_sizes=tensor([2, 1, 1]), sorted_indices=None, unsorted_indices=None)
y_packed:
 PackedSequence(data=tensor([[-0.2084,  0.9807],
        [-0.4690,  0.9106],
        [-0.4862, -0.1909],
        [ 0.6134, -0.6464]], grad_fn=<CatBackward0>), batch_sizes=tensor([2, 1, 1]), sorted_indices=None, unsorted_indices=None)


In [ ]:

print("4.--------------")
y_sort, length = nn_utils.rnn.pad_packed_sequence(y_packed, batch_first=True)
print("sort unpacked output:\n", y_sort)
print(y_sort.shape)

print("5.--------------")
# unsort output to original order
y = torch.index_select(y_sort, dim=0, index=idx_unsort)
print("org unpacked output:\n", y)
print(y.shape)

print("6.--------------")
# unsort output to original order
last_h = torch.index_select(h_n[-1], dim=0, index=idx_unsort)
print("last hidden state:\n", last_h)

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import utils as nn_utils
batch_size = 2
max_length = 3
hidden_size = 2
n_layers =1
 
tensor_in = torch.FloatTensor([[1, 2, 3], [1, 0, 0]]).resize_(2,3,1)
tensor_in = Variable( tensor_in ) #[batch, seq, feature], [2, 3, 1]
seq_lengths = [3,1] # list of integers holding information about the batch size at each sequence step
 
# pack it
pack = nn_utils.rnn.pack_padded_sequence(tensor_in, seq_lengths, batch_first=True)
 
# initialize
rnn = nn.RNN(1, hidden_size, n_layers, batch_first=True)
h0 = Variable(torch.randn(n_layers, batch_size, hidden_size))
 
#forward
out, _ = rnn(pack, h0)
 
# unpack
unpacked = nn_utils.rnn.pad_packed_sequence(out)
print('111',unpacked)

111 (tensor([[[-0.0026, -0.8799],
         [ 0.1316, -0.6966]],

        [[ 0.3797, -0.8921],
         [ 0.0000,  0.0000]],

        [[ 0.5364, -0.9789],
         [ 0.0000,  0.0000]]], grad_fn=<CopySlices>), tensor([3, 1]))
